In [2]:
import sqlite3 as sq3
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import config
import matplotlib.pyplot as plt
%matplotlib inline

conn = sq3.connect(config.mlb_db_path)
event = pd.read_sql(sql='select * from bevent where gameid like "%201%"', con=conn)

event['date'] = event['gameid'].apply(lambda x: int(x[3:]))
cut_point = 201707060
learn, test = event[event['date'] < cut_point], event[event['date'] >= cut_point]
print('Test/Learn Ratio: {}%'.format(round(float(len(test))/float(len(learn)),4)*100))

Test/Learn Ratio: 19.62%


In [21]:
learn['hteam'] = learn['gameid'].apply(lambda x: x[:3])
cv = learn[['gameid','eventnum']].groupby('gameid').max().reset_index()
wl = cv.merge(learn[['gameid','eventnum','hscore','vscore','hteam','vteam']], on=['gameid','eventnum'])[['hscore','vscore','hteam','vteam']]

In [22]:
wl['winner'] = np.where(wl['hscore']>wl['vscore'],wl['hteam'],wl['vteam'])
wl['loser'] = np.where(wl['hscore']<wl['vscore'],wl['hteam'],wl['vteam'])

wl_ = wl[wl['hscore'] != wl['vscore']]

In [23]:
xx = wl_[['winner','loser']]

In [26]:
players = list(set(xx['winner'].tolist()))

In [27]:
players

[u'MIL',
 u'MIN',
 u'TOR',
 u'ATL',
 u'BOS',
 u'OAK',
 u'CIN',
 u'NYN',
 u'FLO',
 u'BAL',
 u'NYA',
 u'COL',
 u'TEX',
 u'MIA',
 u'SEA',
 u'CLE',
 u'PIT',
 u'CHA',
 u'DET',
 u'PHI',
 u'CHN',
 u'SLN',
 u'HOU',
 u'WAS',
 u'LAN',
 u'KCA',
 u'ANA',
 u'SFN',
 u'MON',
 u'TBA',
 u'ARI',
 u'SDN']

In [29]:
import elo

In [30]:
EL = elo.ELO(player_list=players)

In [38]:
for w,l in zip(xx['winner'].tolist(),xx['loser'].tolist()):
    EL.do_competition(winner=w,loser=l)

In [43]:
for k, v in EL.EM.player_elo_cache.items():
    print('{}\t{}'.format(k,v))

CHA	1704.59619922
MIL	1590.34900983
MIN	1496.94314513
TOR	1614.5882904
CLE	1626.7980514
ATL	1493.88854492
BOS	1732.40238846
DET	1599.04260485
CIN	1583.1732476
CHN	1615.90208439
SLN	1624.46282817
HOU	1452.28662923
NYN	1595.17531724
FLO	1698.40498925
LAN	1675.3175419
BAL	1689.96337315
NYA	1586.9853641
WAS	1586.07254569
COL	1587.81937218
OAK	1583.7144332
SFN	1489.93745316
PHI	1619.88809907
KCA	1522.61266344
ANA	1756.02269255
TEX	1606.91684292
MIA	1523.14559196
MON	1629.31310871
SEA	1557.31493354
TBA	1652.9394402
ARI	1588.75073459
SDN	1561.70759163
PIT	1553.56488791
